# __**Running Freesurfer on Argon HPC**__


This guide will cover how we will perform a longitudinal analysis using freesurfer utilizing our university's computing resources. 

__Resources__

We will be talking about multiple resources not directly related to neuroimaging:

-  [Docker](https://www.docker.com/): Provides a way to make/run software with complex dependencies into a standalone independent container (think executable file)


-  [Singularity](http://singularity.lbl.gov/index.html): Similar to Docker, but does not require administrator privledges to run the standalone software; this means we can use it on the cluster where users do not have administrator privledges


-  [HPC-Argon](https://wiki.uiowa.edu/display/hpcdocs/Argon+Cluster): A collection (cluster) of fancy computers that can chug through a bunch of data faster and more intelligently then the computers in our lab.

And a couple resources that are related to neuroimaging:

-  [FreeSurfer](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3685476/): The software workhorse designed to perform longitudinal structural analysis

-  [Brain Imaging Data Structure (BIDS)](http://bids.neuroimaging.io/): The organizational principles our data must follow

-  [BIDS-Apps](http://bids-apps.neuroimaging.io/), specifically the [FreeSurfer BIDS-App](https://github.com/BIDS-Apps/freesurfer): This provides the means to package the freesurfer software into a docker and/or singularity container that we can run anywhere.


__Assumptions__

This tutorial assumes:

1. You have an Argon account
2. You have access to your data via Argon
3. Your data are organized in accordance to the BIDS standard
3. The Singularity container you are using already has been made.
4. You are using either Mac OSX or Linux

## Step 1: Login to Argon
Open up your [terminal](http://linuxcommand.org/) and type the following, replacing *hawkid* with your hawkid

__Generic__

```bash
ssh hawkid@argon.hpc.uiowa.edu
```

__Example__

```bash
ssh jdkent@argon.hpc.uiowa.edu
```


## Step 2: Navigate to the correct Project Directory

In Argon, all shares (places where we store data) are located under the directory `/Shared`. So we will navigate to the correct project directory starting from there

```bash
cd /Shared/vosslabhpc/Projects/PACR-AD/Imaging/BIDS/derivatives/code
```

The above `code` directory will be where we make our *jobs* that we submit to the Argon cluster using our singularity container

## Step 3: Make Appropiate Directories
In the code directory we will make a sub-directory (and several sub-sub-directories):

sub-directory
-  freesurfer_long: where we keep job files and related output in relation to freesurfer

sub-sub-directories
-  jobs: where we store the job files that we will submit to the Argon cluster
-  err: where the error output will go in case something goes wrong with a job
-  out: where the regular output will go from submitting the job

Assuming we are in the code directory, we will make the sub-directory and sub-sub-directories with the following command

```bash
mkdir -p freesurfer_long/{jobs,out,err}
```

__EDIT__:
we also need to make one more directory for the output of freesurfer in `/Shared/vosslabhpc/Projects/PACR-AD/Imaging/BIDS/derivatives/`

```bash
mkdir /Shared/vosslabhpc/Projects/PACR-AD/Imaging/BIDS/derivatives/freesurfer_long
```


## Step 4: Set up a Template Job File
Now that all the directories are in place we need to make a template job file. From this file, we will derive the individual subject job files we will submit to the Argon cluster.

First we cd into the jobs directory:
```bash
cd ./freesurfer_long/jobs
```

Then we make a blank file:
```bash
touch TEMPLATE_freesurfer_long.job
```

Then we open the blank file so that we can write in it:
```bash
nano TEMPLATE_freesurfer_long.job
```

Then we would write something like the following into the blank file:
```bash
#!/bin/bash

#$ -pe smp 16
#$ -q UI
#$ -m bea
#$ -M james-kent@uiowa.edu
#$ -o /Shared/vosslabhpc/Projects/PACR-AD/Imaging/BIDS/derivatives/code/freesurfer_long/out
#$ -e /Shared/vosslabhpc/Projects/PACR-AD/Imaging/BIDS/derivatives/code/freesurfer_long/err
OMP_NUM_THREADS=10
singularity run -H ${HOME}/singularity_home -B /Shared/vosslabhpc:/mnt \
/Shared/vosslabhpc/UniversalSoftware/SingularityContainers/jdkent_freesurfer-2017-11-01-51dcee798e77.img \
/mnt/Projects/PACR-AD/Imaging/BIDS /mnt/Projects/PACR-AD/Imaging/BIDS/derivatives/freesurfer_long \
participant --participant_label SUBJECT \
--n_cpus 16 --skip_bids_validator \
--stages all --multiple_session longitudinal --license_file /mnt/UniversalSoftware/freesurfer_license.txt
```

__Explanation__

To understand what's going on above the `singularity run ...` command, please see [basic job submission](https://wiki.uiowa.edu/display/hpcdocs/Basic+Job+Submission), and [advanced job submission](https://wiki.uiowa.edu/display/hpcdocs/Advanced+Job+Submission)

I'll briefly describe the `-H` and `-B` options displayed after `singularity run`
-  `-H`, this specifies what singularity will treat as the `$HOME` directory. Since Singularity essentially operates on top of your existing environment instead of completely separately (as docker does), sometimes default settings in your regular `$HOME` directory can override the settings in the singularity container. To avoid this conflict, I just make a new empty directory for Singularity to operate in.

-  `-B`, The singularity container, however, does not have access to the all folders on our system, so this option explicitly binds where are data are to the singularity container so that freesurfer software can access and process the data.

For the options specified after the singularity container (`/Shared/vosslabhpc/UniversalSoftware/SingularityContainers/jdkent_freesurfer-2017-11-01-51dcee798e77.img`), I'll refer you back to the [freesurfer BIDS-App page](https://github.com/BIDS-Apps/freesurfer)

## Step 5: Generate all the subject specific job files
In the Template file you may have noticed the keyword __SUBJECT__ specified after the option `--participant-label`. To generate all the subject specific files we will use [sed](http://www.grymoire.com/Unix/Sed.html) to replace __SUBJECT__ with our specific participant labels (e.g. ControlGE140).

Since we are lazy and don't want to write out our sed commands, we will call a script called generate_jobs.sh in a separate directory:
```bash
cd /Shared/vosslabhpc/Projects/PACR-AD/Imaging/BIDS/derivatives/code/misc
```

In this directory we will see a script called generate_jobs.sh and it contains the following code:
```bash
#!/usr/bin/env bash

function HelpMe
{
  echo "Usage:"
  echo "generate_jobs.sh -i SourceData -o CodeDir -t Template"
  echo "SourceData = Directory containing source data"
  echo "CodeDir = Output directory to place job scripts"
  echo "Template = The template file containing the skeleton script"
  echo "cont. the skeleton script should have SUBJECT in the place of participant label"
}


while getopts "i:o:t:h" OPTION; do
    case ${OPTION} in
        i)
            SourceData=${OPTARG}
            ;;
        o)
            CodeDir=${OPTARG}
            ;;
        t)
            Template=${OPTARG}
            ;;
        h)
            HelpMe
            exit 0
            ;;
    esac
done

SourceData=${SourceData:-"unset"}
CodeDir=${CodeDir:-"unset"}
Template=${Template:-"unset"}

VarArray=(${SourceData} ${CodeDir} ${Template})
if [[ ${VarArray[@]} =~ "unset" ]]; then
    echo "one of the mandatory variables is not set"
    HelpMe
    exit 1
fi

#get suject list
Subjects=$(find ${SourceData} -mindepth 1 -maxdepth 1 -name "sub-*" | xargs -I {} basename {} | awk -F"-" '{print $2}')

#make job script based on Template
for Subject in ${Subjects[@]}; do
    sed -e "s/SUBJECT/${Subject}/g" ${Template} > ${CodeDir}/sub-${Subject}.job
done
```
__Usage__

Then calling the above (assuming we are in the `misc`) directory we will get the subject specific job files:

```bash
./generate_jobs.sh -i ../../../ -o ../freesurfer_long/jobs -t ../freesurfer_long/jobs/TEMPLATE_freesurfer_long.job
```


## Step 6: Submit ONE of the generated jobs to the cluster
It's always important to test if your job scripts work; if there's a systematic error in all the job files, you don't want to waste your time waiting for all of them to error out, so we will always test one job first.

Go back to the jobs directory:
```bash
cd ../freesurfer_long/jobs
```

then submit ONE job to the cluster:
```bash
qsub sub-controlGE140.job
```

Then periodically check the status of the job, and the err/out files
```bash
# check status
qstat -u jdkent
# check err
cat ../err/sub-controlGE140.e.*
# check out
cat ../out/sub-controlGE140.o.*
```
If all appears well after a while, then you can submit all the remaining jobs to the cluster

## Step 7: Submit the rest of the jobs
Now we are ready to submit the rest of the jobs to the cluster.

Since we already submitted `sub-controlGE140` to the cluster we will change their job file name so we don't accidently submit their file again.
```bash
mv sub-controlGE140.job test_sub-controlGE140.job
```

Then we use a `for loop` to submit the rest of the jobs to the cluster:
```bash
for job in sub-*.job; do qsub ${job}; done
```

Then you can check to see if everything is going well:
```bash
qstat -u jdkent
```


## CONGRATS, FINISHED!
Now you just wait forever for freesurfer to finish